DATASET TRANSFORMS

'''
Transforms can be applied to PIL images, tensors, ndarrays, or custom data
during creation of the DataSet

complete list of built-in transforms: 
https://pytorch.org/docs/stable/torchvision/transforms.html

On Images
---------
CenterCrop, Grayscale, Pad, RandomAffine
RandomCrop, RandomHorizontalFlip, RandomRotation
Resize, Scale

On Tensors
----------
LinearTransformation, Normalize, RandomErasing

Conversion
----------
ToPILImage: from tensor or ndrarray
ToTensor : from numpy.ndarray or PILImage

Generic
-------
Use Lambda 

Custom
------
Write own class

Compose multiple Transforms
---------------------------
composed = transforms.Compose([Rescale(256),
                               RandomCrop(224)])
'''

In [1]:
import torch
import torchvision
from torch.utils.data import Dataset
import numpy as np

In [2]:
class WineDataset(Dataset):
    
    def __init__(self,transform=None):
        xy = np.loadtxt('wine.csv', delimiter=",", dtype=np.float32, skiprows=1)
        self.n_samples = xy.shape[0]
        
        #note that we do not convert to tensor here,so x_data and y_data is only numpy array
        self.x_data = xy[:,1:]
        self.y_data = xy[:,[0]]
        
        self.transform = transform
        
    def __getitem__(self,index):
        self.sample = self.x_data[index],self.y_data[index]
        
        if self.transform:
            self.sample = self.transform(self.sample)
            
        return self.sample
    
    def __len__(self):
        return self.n_samples
    

In [4]:
#Custom Transforms
#implement __call__(self,sample)

class ToTensor:
    #convert ndarrays to Tensors
    def __call__(self,sample):
        self.inputs,self.targets = sample
        return torch.from_numpy(self.inputs), torch.from_numpy(self.targets)
    
    
class MulTransform:
    #multiply inputs with a given factor
    def __init__(self,factor):
        self.factor = factor
        
    def __call__(self,sample):
        self.inputs,self.targets = sample
        self.inputs *= self.factor
        return self.inputs,self.targets

In [5]:
print('Without Transform')
dataset = WineDataset()
first_data = dataset[0]
features,labels = first_data
print(type(features),type(labels))
print(features,labels)

Without Transform
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
[1.423e+01 1.710e+00 2.430e+00 1.560e+01 1.270e+02 2.800e+00 3.060e+00
 2.800e-01 2.290e+00 5.640e+00 1.040e+00 3.920e+00 1.065e+03] [1.]


In [6]:
print('With Tensor Transform')
dataset = WineDataset(transform=ToTensor())
first_data = dataset[0]
features,labels = first_data
print(type(features),type(labels))
print(features,labels)

With Tensor Transform
<class 'torch.Tensor'> <class 'torch.Tensor'>
tensor([1.4230e+01, 1.7100e+00, 2.4300e+00, 1.5600e+01, 1.2700e+02, 2.8000e+00,
        3.0600e+00, 2.8000e-01, 2.2900e+00, 5.6400e+00, 1.0400e+00, 3.9200e+00,
        1.0650e+03]) tensor([1.])


In [7]:
print('\nWith Tensor and Multiplication Transform')
composed = torchvision.transforms.Compose([ToTensor(),MulTransform(4)]) #list of transforms
dataset = WineDataset(transform=composed)
first_data = dataset[0]
features,labels = first_data
print(type(features),type(labels))
print(features,labels)


With Tensor and Multiplication Transform
<class 'torch.Tensor'> <class 'torch.Tensor'>
tensor([5.6920e+01, 6.8400e+00, 9.7200e+00, 6.2400e+01, 5.0800e+02, 1.1200e+01,
        1.2240e+01, 1.1200e+00, 9.1600e+00, 2.2560e+01, 4.1600e+00, 1.5680e+01,
        4.2600e+03]) tensor([1.])


There are built-in datasets and built-in transforms in PyTorch

In [9]:
dataset = torchvision.datasets.MNIST(root='./data',download=True,transform=torchvision.transforms.ToTensor())

100%|██████████| 9912422/9912422 [00:08<00:00, 1182957.64it/s]


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw



100%|██████████| 28881/28881 [00:00<00:00, 661715.88it/s]


Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw



100%|██████████| 1648877/1648877 [00:00<00:00, 1919709.42it/s]


Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw



100%|██████████| 4542/4542 [00:00<?, ?it/s]

Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw

